In [1]:
# 한글 폰트 설치하기 (꼭! 설치가 완료되면 [런타임 다시 시작]을 누르고 다시 실행하기)
# !apt install fonts-nanum -y

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# # 한글 폰트 설정하기
# fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# font = fm.FontProperties(fname=fontpath, size=10)
# plt.rc('font', family='NanumBarunGothic')
# matplotlib.font_manager._rebuild()
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

from efficientnet_pytorch import EfficientNet

In [2]:

from matplotlib import font_manager, rc
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
mpl.rc('font', family='Malgun Gothic')


In [3]:
# 필요한 라이브러리 설치하기
!git clone https://github.com/ndb796/bing_image_downloader

fatal: destination path 'bing_image_downloader' already exists and is not an empty directory.


In [4]:
import os
import shutil
from bing_image_downloader.bing_image_downloader import downloader


directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

# 초기 디렉토리 만들기
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)

# 수집한 이미지를 학습 데이터와 평가 데이터로 구분하는 함수
def dataset_split(query, train_cnt):
    # 학습 및 평가 데이터셋 디렉토리 만들기
    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)
    # 학습 및 평가 데이터셋 준비하기
    cnt = 0
    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/train/' + query + '/' + file_name)
        else:
            print(f'[Test Dataset] {file_name}')
            shutil.move(query + '/' + file_name, './custom_dataset/test/' + query + '/' + file_name)
        cnt += 1
    shutil.rmtree(query)

In [5]:
# query = '마동석'
# downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
# dataset_split(query, 30)

In [6]:
# query = '김종국'
# downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
# dataset_split(query, 30)

In [7]:
# query = '이병헌'
# downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
# dataset_split(query, 30)

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import time

In [9]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms_train = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(), # 데이터 증진(augmentation)
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

transforms_test = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=4, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=4, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

학습 데이터셋 크기: 90
테스트 데이터셋 크기: 30
클래스: ['김종국', '마동석', '이병헌']


In [10]:
# from PIL import Image

# # Load and preprocess the image
# def preprocess(image_path):
#     # Define the transformations to be applied
#     transform = transforms.Compose([
#         transforms.Resize((512, 512)),              # Resize the image to a specific size
#         transforms.ToTensor(),                       # Convert the image to a tensor
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
#     ])

#     # Load the image using PIL
#     image = Image.open(image_path)

#     # Apply the transformations
#     preprocessed_image = transform(image)

#     # Add a batch dimension to match the model's input shape
#     preprocessed_image = preprocessed_image.unsqueeze(0)

#     return preprocessed_image

# # # Example usage
# image_path = './custom_dataset/train'  # Replace with the path to your image
# preprocessed_image = preprocess_image(image_path)

# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'))
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'))

In [11]:
# preprocessed_image_train = preprocess(train_datasets)

In [12]:
# # Open the image using PIL
# image = Image.open('./custom_dataset/train/김종국/image_1.jpg')

# # Display image properties
# print("Image size:", image.size)
# print("Image mode:", image.mode)

# # Perform further operations on the image as needed
# # ...

# # Close the image after you're done
# image.close()

In [14]:
# model = models.resnet34(pretrained=True)
model = EfficientNet.from_name('efficientnet-b0', num_classes=3)
# num_features = model.fc.in_features
# 전이 학습(transfer learning): 모델의 출력 뉴런 수를 3개로 교체하여 마지막 레이어 다시 학습
# model.fc = nn.Linear(num_features, 3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 50
model.train()
start_time = time.time()

# 전체 반복(epoch) 수 만큼 반복하며
for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    # 배치 단위로 학습 데이터 불러오기
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 모델에 입력(forward)하고 결과 계산
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    # 학습 과정 중에 결과 출력
    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#0 Loss: 1.1968 Acc: 40.0000% Time: 116.4613s
#1 Loss: 0.9491 Acc: 51.1111% Time: 233.1682s
#2 Loss: 1.0169 Acc: 45.5556% Time: 347.5349s
#3 Loss: 0.9340 Acc: 57.7778% Time: 461.3186s
#4 Loss: 0.9776 Acc: 53.3333% Time: 575.7684s
#5 Loss: 0.9929 Acc: 53.3333% Time: 689.6529s
#6 Loss: 0.8695 Acc: 62.2222% Time: 803.2070s
#7 Loss: 0.7996 Acc: 61.1111% Time: 917.1191s
#8 Loss: 0.7389 Acc: 67.7778% Time: 1031.1025s
#9 Loss: 1.0237 Acc: 51.1111% Time: 1144.8861s
#10 Loss: 0.9520 Acc: 52.2222% Time: 1258.9689s
#11 Loss: 0.9086 Acc: 61.1111% Time: 1373.0194s
#12 Loss: 0.9029 Acc: 56.6667% Time: 1486.6856s
#13 Loss: 0.8199 Acc: 60.0000% Time: 1600.4530s
#14 Loss: 0.8370 Acc: 62.2222% Time: 1715.3526s
#15 Loss: 0.9374 Acc: 61.1111% Time: 1829.0531s
#16 Loss: 0.7889 Acc: 62.2222% Time: 1943.0195s
#17 Loss: 0.9240 Acc: 56.6667% Time: 2056.8862s
#18 Loss: 0.7834 Acc: 64.4444% Time: 2171.2858s
#19 Loss: 0.7513 Acc: 62.2222% Time: 2285.5864s
#20 Loss: 0.8360 Acc: 64.4444% Time: 2400.1693s
#21 Loss: 

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in test_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
        print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
        imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])

    epoch_loss = running_loss / len(test_datasets)
    epoch_acc = running_corrects / len(test_datasets) * 100.
    print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))